In [3]:
import os
import numpy as np
import pandas as pd
import pickle
from scipy.optimize import least_squares

Load the results from HALS generated with "04_HALS_atm_baro.ipynb" and "04_HALS_head_baro.ipynb"


In [4]:
# Load results

path = 'Results/pkl_files/'
    
with open(path + 'baro.pkl', 'rb') as f:
    baro = pickle.load(f)
    
with open(path + 'strain_to_atm_r.pkl', 'rb') as f:
    strain = pickle.load(f)

with open(path + 'head_baro.pkl', 'rb') as f:
    head_baro = pickle.load(f)

Decoupling between M2 and S2

In [5]:
def S2_dis(Z_GW_S2, Z_GW_M2, Z_ET_S2, Z_ET_M2):
    params = Z_GW_S2 - (Z_GW_M2 / Z_ET_M2) * Z_ET_S2
    return params

def R2P(x):
    return abs(x), np.angle(x)

Z_GW_S2_B1 = head_baro['complex']['B1'].loc['S2']
Z_GW_M2_B1 = head_baro['complex']['B1'].loc['M2']

Z_ET_S2_B1 = strain['complex']['B1'].loc['S2']
Z_ET_M2_B1 = strain['complex']['B1'].loc['M2']

Z_GW_S2_B2 = head_baro['complex']['B2'].loc['S2']
Z_GW_M2_B2 = head_baro['complex']['B2'].loc['M2']

Z_ET_S2_B2 = strain['complex']['B2'].loc['S2']
Z_ET_M2_B2 = strain['complex']['B2'].loc['M2']

S2_dis_B1 = S2_dis(Z_GW_S2_B1, Z_GW_M2_B1, Z_ET_S2_B1, Z_ET_M2_B1)
S2_dis_B2 = S2_dis(Z_GW_S2_B2, Z_GW_M2_B2, Z_ET_S2_B2, Z_ET_M2_B2)

Z_AT_GW_B1 = R2P(S2_dis_B1)
Z_AT_GW_B2 = R2P(S2_dis_B2)

head_baro['amp']['B1']['S2'] = Z_AT_GW_B1[0]
head_baro['phase']['B1']['S2'] = Z_AT_GW_B1[1]

head_baro['amp']['B2']['S2'] = Z_AT_GW_B2[0]
head_baro['phase']['B2']['S2'] = Z_AT_GW_B2[1]

In [6]:
# Merge in one dic
baro['amp_baro'] = baro.pop('amp')
baro['phase_baro'] = baro.pop('phase')

head_baro['amp_head'] = head_baro.pop('amp') 
head_baro['phase_head'] = head_baro.pop('phase')

strain['amp_str'] = strain.pop('amp') 
strain['phase_str'] = strain.pop('phase')

data = {**head_baro, **baro}

data.keys()

dict_keys(['complex', 'amp_head', 'phase_head', 'amp_baro', 'phase_baro'])

Compute phase amplitude ratio and phase shift


In [7]:
amp_ratio = (data['amp_head']) / (data['amp_baro']*1E3/1E4) # Remeber, this is barometric efficiency and 
#pressure in Pa

phase_shift = np.rad2deg(data['phase_head']-data['phase_baro'])


Now we call the new anayltical solution of this work

In [8]:
from scipy.special import kv

# Functions
def omega_fn(PERIOD):
    params = 2 * np.pi / PERIOD
    return params

def betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ):
    params = ((K_LE / (K_AQ * B_AQ * B_LE)) + ((1j * omega * S_AQ * B_AQ) / (K_AQ * B_AQ))) ** 0.5
    return params

def argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE):
    params = (1j * omega * B_AQ) / (1j * omega * S_AQ * B_AQ + K_LE / B_LE)
    return params

def tide_fn(P_0, B):
    params = ((P_0) / (3 * B)) # P_0 in metters and B in Pa so 10^4 to change units

    return params

def xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta):
    params = 1 + ((1j * omega * R_W) / (2 * K_AQ * B_AQ * betta)) * (kv(0, betta * R_W) / kv(1, betta * R_W)) * (R_C / R_W)**2
    return params

def h_w_fn(argument, tide, xi):
    params = argument * tide / xi
    return params

def drawdown_fn(omega, R_C, h_w, betta, K_AQ, B_AQ, R):
    params = -(1j * omega * R_C ** 2 * h_w * kv(0, betta * R)) / (2 * K_AQ * B_AQ * betta * R_W * kv(1, betta * R))
    return params

def flux_fn(omega, h_w,  R_C):
    params = omega * np.absolute(h_w) * np.pi * R_C**2 *1E3
    return params

def Ss_stress_fn(S_E, B):
    params = S_E + 1E4 / B
    return params

def this_work(K_AQ, S_E, K_LE, B):
    
    S_AQ = Ss_stress_fn(S_E, B)
    omega = omega_fn(PERIOD)
    betta = betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ)
    argument = argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE)
    tide = tide_fn(P_0, B)
    xi = xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta)
    h_w = h_w_fn(argument, tide, xi)
    
    amp = (-P_0 / (B * S_AQ) + (P_0 * 1E-4) + h_w) / (P_0 * 1E-4)
    shift = np.angle((-P_0 / (B * S_AQ) + (P_0 * 1E-4) + h_w) / (P_0 * 1E-4), deg=True)
    
    return amp.real, shift.real

def inv_this_work(vars):
    
    K_AQ, S_E, K_LE, B = vars
    
    S_AQ = Ss_stress_fn(S_E, B)
    omega = omega_fn(PERIOD)
    betta = betta_fn(K_LE, K_AQ, B_AQ, B_LE, omega, S_AQ)
    argument = argument_fn(omega, S_AQ, B_AQ, K_LE, B_LE)
    tide = tide_fn(P_0, B)
    xi = xi_fn(R_W, R_C, omega, K_AQ, B_AQ, betta)
    h_w = h_w_fn(argument, tide, xi)
    
    amp = (-P_0 / (B * S_AQ) + (P_0 * 1E-4) + h_w) / (P_0 * 1E-4)
    shift = np.angle((-P_0 / (B * S_AQ) + (P_0 * 1E-4) + h_w) / (P_0 * 1E-4), deg=True)
    
    amp = 1-amp
    shift = 1-shift
    
    opt_amplitude = float((amp.real - amp_obs)/amp_obs)
    opt_shift = float((shift.real - shift_obs)/shift_obs)
    
    FO = (opt_amplitude) + (opt_shift)
    
    return [FO]

In [14]:
# These are the borehole geometry properties

r_w = np.array([156, 203]) * 1E-3 / 2 # radius
r_c = r_w # radius of the case
#b_aq = np.array([4, 6]) # aquifer depth
b_aq = np.array([12, 28]) # aquifer depth
b_le = np.array([25, 28]) # aquitard depth
F = np.ones((5)) * 1.97322 # frequency of the M2 tide
period = 1/F * 24 * 3600 # period of the M2 tide
p_0 = data['amp_baro'].iloc[[1]].values

# Now we choose which borehole we would like to study!!

borehole = 0 # 0 = B1

amp_obs = amp_ratio.iloc[1][borehole]
shift_obs = phase_shift.iloc[1][borehole]

R_W = r_w[borehole]
R_C = r_c[borehole]
PERIOD = period[borehole]
B_AQ = b_aq[borehole]
B_LE = b_le[borehole]
E_0 = 1E-7 # Signal amplitude [-] (this amplitude is unrealistic, but convinient for this example)
P_0 = p_0[0][borehole] * 1E3


In [15]:
Earth_K_AQ = np.array([1.1E-5,1.0E-4])
Earth_S_E = np.array([1.8E-6,5E-7])
Earth_K_LE = np.array([5.4E-10,6E-8])

###### ---                                  --- ######

K_AQ_init = Earth_K_AQ[borehole]
S_E_init = Earth_S_E[borehole]
K_LE_init = Earth_K_LE[borehole]

#B1
B_init = 2E8

#B2
#B_init = 1E10

# Initial conditions

K_AQ_MAX = K_AQ_init * 1.5
S_E_MAX = S_E_init * 1.5
K_LE_MAX = K_LE_init * 1.5
B_MAX = B_init * 1.5

K_AQ_MIN = K_AQ_init *.5
S_E_MIN = S_E_init *.5
K_LE_MIN = K_LE_init *.5
B_MIN = B_init *.5

In [17]:
# For B1
P = least_squares(inv_this_work, (K_AQ_init, S_E_init, K_LE_init, B_init), jac = '3-point', bounds=([K_AQ_MIN, 4E-7, K_LE_MIN, B_MIN], [K_AQ_MAX, 1.8E-6, K_LE_MAX, B_MAX]))

# For B2
#P = least_squares(inv_this_work, (K_AQ_init, S_E_init, K_LE_init, B_init))#, jac = '3-point', bounds=([K_AQ_MIN, 4E-7, K_LE_MIN, B_MIN], [K_AQ_MAX, 1.8E-6, K_LE_MAX, B_MAX]))

print('Aquifer hydraulic conductivity: ', P.x[0])
print('Specific storage at constant strain: ', P.x[1])
print('Aquitard hydraulic conductivity: ', P.x[2])
print('Bulk modulus of the aquifer: ', P.x[3])

print('----------')

found_res = this_work(P.x[0], P.x[1], P.x[2], P.x[3])

print('Found ampliturde ratio: ', found_res[0])
print('Real ampliturde ratio: ', amp_obs)

print('Found phase shift: ', found_res[1])
print('Real phase shift: ', shift_obs)

print('----------')

# Compute error of the search, i.e, final value of objetive function

obj_amplitude = float(np.abs(found_res[0] - amp_obs)/amp_obs)
obj_shift = float(np.abs(found_res[1] - shift_obs)/shift_obs)
    
obj_FO = np.abs(obj_amplitude) + np.abs(obj_shift)

print('Goodness of the search: ', obj_FO)

Aquifer hydraulic conductivity:  1.6293530156854887e-05
Specific storage at constant strain:  1.7999962499086762e-06
Aquitard hydraulic conductivity:  8.099493120385284e-10
Bulk modulus of the aquifer:  299388684.0778157
----------
Found ampliturde ratio:  0.36680133281754923
Real ampliturde ratio:  0.822481356305684
Found phase shift:  -0.7105842690667752
Real phase shift:  -0.716769787895686
----------
Goodness of the search:  0.5626605386892811
